## Trabajo Grupal 6

      Integrantes Grupo 8:
    
    - Renzo Mosquera (20181960)
    - Yenner Huancahuire (20173340)
    - Pamela Obregón (20173040)

In [1]:
# Importamos librerias

import pandas as pd
import numpy as np
import warnings
import pyreadstat
import seaborn as sns
import re 
import matplotlib.pyplot as plt 
from matplotlib.lines import Line2D
warnings.filterwarnings('ignore')

In [2]:
# Importamos las bases de datos (data.dta y unidos.dta)

data = pd.read_stata("../../data/Juntos_program/data.dta",
                        convert_categoricals=False)

unidos = pd.read_stata("../../data/Juntos_program/unidos.dta",
                        convert_categoricals=False)

In [3]:
# Mostramos "data"

data

,numper,numpanh15,numpanh16,numpanh17,numpanh18,numpanh19,mes_15,ubigeo_15,dominio_15,p400a3_15,...,mes_18,ubigeo_18,dominio_18,p400a3_18,p4022_18,mes_19,ubigeo_19,dominio_19,p400a3_19,p4022_19
0,15368,1158,1158,1158,1158,1158,11,10101,sierra norte,1946.0,...,12,10101,sierra norte,1946.0,0.0,11,10101,sierra norte,1946.0,0.0
1,15369,1158,1158,1158,1158,1158,11,10101,sierra norte,1973.0,...,12,10101,sierra norte,1973.0,0.0,11,10101,sierra norte,1973.0,0.0
2,15380,1162,1162,1162,1162,1162,11,10101,sierra norte,1987.0,...,12,10101,sierra norte,1987.0,0.0,11,10101,sierra norte,1987.0,1.0
3,15381,1162,1162,1162,1162,1162,11,10101,sierra norte,2009.0,...,12,10101,sierra norte,2009.0,0.0,11,10101,sierra norte,2009.0,0.0
4,15410,1185,1185,1185,1185,1185,7,10101,sierra norte,1955.0,...,7,10101,sierra norte,1955.0,0.0,7,10101,sierra norte,1955.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5541,34848,18549,18549,18549,18549,18549,12,250101,selva,2002.0,...,12,250101,selva,2002.0,0.0,10,250101,selva,2002.0,1.0
5542,34846,18549,18549,18549,18549,18549,12,250101,selva,1985.0,...,12,250101,selva,1985.0,0.0,10,250101,selva,1985.0,0.0
5543,34847,18549,18549,18549,18549,18549,12,250101,selva,1976.0,...,12,250101,selva,1976.0,0.0,10,250101,selva,1976.0,1.0
5544,34849,18549,18549,18549,18549,18549,12,250101,selva,2007.0,...,12,250101,selva,2007.0,0.0,10,250101,selva,2007.0,1.0


In [4]:
# Mostramos los nombres de las variables de la base "data"

data.columns

Index(['numper', 'numpanh15', 'numpanh16', 'numpanh17', 'numpanh18',
       'numpanh19', 'mes_15', 'ubigeo_15', 'dominio_15', 'p400a3_15',
       'p4022_15', 'mes_16', 'ubigeo_16', 'dominio_16', 'p400a3_16',
       'p4022_16', 'mes_17', 'ubigeo_17', 'dominio_17', 'p400a3_17',
       'p4022_17', 'mes_18', 'ubigeo_18', 'dominio_18', 'p400a3_18',
       'p4022_18', 'mes_19', 'ubigeo_19', 'dominio_19', 'p400a3_19',
       'p4022_19'],
      dtype='object')

In [5]:
## Transformación de formato wide a long

# Se usará la variable "numper" como indentificador.
# Renombramos la variable "numpanh"

data.rename(columns = {"numpanh15":"numpanh_15", "numpanh16":"numpanh_16", 
                       "numpanh17":"numpanh_17", "numpanh18":"numpanh_18",
                      "numpanh19":"numpanh_19"}, inplace = True)
data.columns

Index(['numper', 'numpanh_15', 'numpanh_16', 'numpanh_17', 'numpanh_18',
       'numpanh_19', 'mes_15', 'ubigeo_15', 'dominio_15', 'p400a3_15',
       'p4022_15', 'mes_16', 'ubigeo_16', 'dominio_16', 'p400a3_16',
       'p4022_16', 'mes_17', 'ubigeo_17', 'dominio_17', 'p400a3_17',
       'p4022_17', 'mes_18', 'ubigeo_18', 'dominio_18', 'p400a3_18',
       'p4022_18', 'mes_19', 'ubigeo_19', 'dominio_19', 'p400a3_19',
       'p4022_19'],
      dtype='object')

In [7]:
# Creamos la lista de variables que estarán en la base de formato "long"

varlong = list( map( lambda x: re.sub('.{3}$',"", x), list(data.columns)[1::]))

# Se eliminan los duplicados

new_var = list(dict.fromkeys(varlong))

new_var

['numpanh', 'mes', 'ubigeo', 'dominio', 'p400a3', 'p4022']

In [8]:
# Realizamos el reshape

reshape_data = pd.wide_to_long(data, stubnames = new_var, i = ['numper'] , 
                                j = 'period' , sep = '_').reset_index()

In [9]:
reshape_data

,numper,period,numpanh,mes,ubigeo,dominio,p400a3,p4022
0,15368,15,1158,11,10101,sierra norte,1946.0,0.0
1,15369,15,1158,11,10101,sierra norte,1973.0,0.0
2,15380,15,1162,11,10101,sierra norte,1987.0,1.0
3,15381,15,1162,11,10101,sierra norte,2009.0,1.0
4,15410,15,1185,7,10101,sierra norte,1955.0,0.0
...,...,...,...,...,...,...,...,...
27725,34848,19,18549,10,250101,selva,2002.0,1.0
27726,34846,19,18549,10,250101,selva,1985.0,0.0
27727,34847,19,18549,10,250101,selva,1976.0,1.0
27728,34849,19,18549,10,250101,selva,2007.0,1.0


In [10]:
# Ordenamos las variables

cols = reshape_data.columns.tolist()

order = [ cols[-4] ] + cols[0:3] + cols[3:-4] + cols[5:8]

reshape_data = reshape_data.loc[:, order]

reshape_data

,ubigeo,numper,period,numpanh,mes,dominio,p400a3,p4022
0,10101,15368,15,1158,11,sierra norte,1946.0,0.0
1,10101,15369,15,1158,11,sierra norte,1973.0,0.0
2,10101,15380,15,1162,11,sierra norte,1987.0,1.0
3,10101,15381,15,1162,11,sierra norte,2009.0,1.0
4,10101,15410,15,1185,7,sierra norte,1955.0,0.0
...,...,...,...,...,...,...,...,...
27725,250101,34848,19,18549,10,selva,2002.0,1.0
27726,250101,34846,19,18549,10,selva,1985.0,0.0
27727,250101,34847,19,18549,10,selva,1976.0,1.0
27728,250101,34849,19,18549,10,selva,2007.0,1.0


In [11]:
# Mostramos "unidos"

unidos

,ubigeo,distrito,provincia,region,unidos
0,10101,Chachapoyas,Chachapoyas,Amazonas,1
1,10102,Asuncion,Chachapoyas,Amazonas,1
2,10103,Balsas,Chachapoyas,Amazonas,0
3,10104,Cheto,Chachapoyas,Amazonas,1
4,10105,Chiliquin,Chachapoyas,Amazonas,0
...,...,...,...,...,...
1869,250302,Irazola,Padre Abad,Ucayali,0
1870,250303,Curimana,Padre Abad,Ucayali,0
1871,250304,Neshuya,Padre Abad,Ucayali,0
1872,250305,Alexander von Humboldt,Padre Abad,Ucayali,0


In [13]:
## Realizamos el Merge Dataset

# Tomamos como data master la base "unidos", ya que tiene como identificador
# la variable ubigeo (sin repetir).

pd.merge(unidos, reshape_data, how = "inner", on = "ubigeo", validate = "1:m")

,ubigeo,distrito,provincia,region,unidos,numper,period,numpanh,mes,dominio,p400a3,p4022
0,10101,Chachapoyas,Chachapoyas,Amazonas,1,15368,15,1158,11,sierra norte,1946.0,0.0
1,10101,Chachapoyas,Chachapoyas,Amazonas,1,15369,15,1158,11,sierra norte,1973.0,0.0
2,10101,Chachapoyas,Chachapoyas,Amazonas,1,15380,15,1162,11,sierra norte,1987.0,1.0
3,10101,Chachapoyas,Chachapoyas,Amazonas,1,15381,15,1162,11,sierra norte,2009.0,1.0
4,10101,Chachapoyas,Chachapoyas,Amazonas,1,15410,15,1185,7,sierra norte,1955.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
27725,250301,Padre Abad,Padre Abad,Ucayali,0,32531,19,16739,2,selva,2013.0,0.0
27726,250301,Padre Abad,Padre Abad,Ucayali,0,32529,19,16739,2,selva,2003.0,0.0
27727,250301,Padre Abad,Padre Abad,Ucayali,0,32530,19,16739,2,selva,2009.0,0.0
27728,250301,Padre Abad,Padre Abad,Ucayali,0,32528,19,16739,2,selva,1985.0,1.0
